In [2]:
! pip install control

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\migue\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# **Miguel Angel Alvarez Guzman 1000871173**

## **Parte 1:**
### Resolver Numerales
- 5.1
# Modelo de FT sin tiempo muerto

La función de transferencia del sistema es:


$ G(s) = \frac{Y(s)}{U(s)} = \frac{b_0}{(t_1 s+1)(t_2 s+1)} $



---

# Modelo de Espacio de Estados (EE) sin tiempo muerto


\begin{aligned}
\dot{x}_1(t) &= -\frac{1}{t_1}\,x_1(t) + \frac{1}{t_1}\,u(t),\\[1mm]
\dot{x}_2(t) &= -\frac{1}{t_2}\,x_2(t) + \frac{1}{t_2}\,x_1(t),\\[1mm]
y(t) &= b_0\,x_2(t).
\end{aligned}


- 5.3.4





In [8]:
import control
import numpy as np

# Definir la función de transferencia:
num = [2, 3]       # Numerador: 2s + 3
den = [1, 4, 5]    # Denominador: s^2 + 4s + 5
sys = control.TransferFunction(num, den)

# 1. Ceros del sistema:
# Se obtienen como las raíces del numerador.
system_zeros = control.zeros(sys)
print("Ceros del sistema:", system_zeros)
# Resultado esperado: array([-1.5])
# (ya que 2s+3=0  => s = -3/2 = -1.5)

# 2. Polos del sistema:
# Se obtienen como las raíces del denominador.
system_poles = control.poles(sys)
print("Polos del sistema:", system_poles)
# Resultado esperado: array([-2.+1.j, -2.-1.j])
# (las raíces de s^2+4s+5=0)

# 3. Ganancia DC del sistema:
# Se evalúa G(s) en s=0.
dc_gain = control.dcgain(sys)
print("Ganancia DC del sistema:", dc_gain)
# Resultado esperado: 3/5 = 0.6
# (ya que G(0) = (2*0+3)/(0^2+4*0+5) = 3/5)

# 4. Valores propios del sistema:
# Se obtiene la representación en espacio de estados y se calcula los eigenvalores de la matriz A.
ss_sys = control.tf2ss(sys)
eigenvalues = np.linalg.eigvals(ss_sys.A)
print("Valores propios del sistema:", eigenvalues)
# Nota: en un sistema minimal los valores propios de A coinciden con los polos.
# Resultado esperado: array([-2.+1.j, -2.-1.j])

# 5. Polinomio característico del sistema:
# Se puede obtener a partir de los polos (que son las raíces del polinomio).
char_poly = np.poly(system_poles)
print("Polinomio característico del sistema:", char_poly)
# Resultado esperado: array([1, 4, 5])


Ceros del sistema: [-1.5+0.j]
Polos del sistema: [-2.+1.j -2.-1.j]
Ganancia DC del sistema: 0.6000000000000001
Valores propios del sistema: [-2.+1.j -2.-1.j]
Polinomio característico del sistema: [1. 4. 5.]


- 5.5.1

In [ ]:
import control as ctl

# Supongamos que Gc y Gptfm ya están definidos como:
Kc = 2.0
tau_c = 1.0
Gc = ctl.TransferFunction([Kc], [tau_c, 1])

# Ejemplo: definir la planta Gptfm(s) = Kp/(τp*s + 1)
Kp = 1.0
tau_p = 3.0
Gptfm = ctl.TransferFunction([Kp], [tau_p, 1])



# Transferencia desde E(s) hasta Y(s)
Yr = ctl.series(Gptfm, Gc)

# Transferencia desde P(s) hasta Y(s)
Yp = Gptfm

print("Yr(s) =", Yr)
print("Yp(s) =", Yp)

Gref(s) (espacio de estado) = <StateSpace>: sys[21]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']
States (2): ['x[0]', 'x[1]']

A = [[-1. -1.]
     [ 1. -2.]]

B = [[1.]
     [0.]]

C = [[0. 1.]]

D = [[0.]]

Gper(s) (espacio de estado) = <StateSpace>: sys[22]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']
States (2): ['x[0]', 'x[1]']

A = [[-3. -3.]
     [ 1.  0.]]

B = [[1.]
     [0.]]

C = [[1. 1.]]

D = [[0.]]



- 5.6.1

In [14]:
import control as ctl

# Definir (o cargar) las funciones de transferencia del controlador y la planta.
# Ejemplo genérico:
numGc = [1]      # Numerador de Gc(s)
denGc = [1, 1]   # Denominador de Gc(s) -> Gc(s) = 1/(s+1)
Gc = ctl.TransferFunction(numGc, denGc)

numGp = [1]      # Numerador de Gptfm(s)
denGp = [1, 2]   # Denominador de Gptfm(s) -> Gptfm(s) = 1/(s+2)
Gptfm = ctl.TransferFunction(numGp, denGp)

# Convertirlas a espacio de estado:
Gc_ss = ctl.ss(Gc)
Gptfm_ss = ctl.ss(Gptfm)

# 1) Función de transferencia desde R(s) hasta Y(s): Gref(s)
#    = [Gc(s)*Gptfm(s)] / [1 + Gc(s)*Gptfm(s)]
# En la librería control, se obtiene con feedback(series(Gc, Gptfm), 1)
# pero usando las representaciones en EE (espacio de estado).
Tloop_ss = ctl.series(Gc_ss, Gptfm_ss)  # Lazo abierto
Gref_ss = ctl.feedback(Tloop_ss, 1)     # Cierra el lazo

# 2) Función de transferencia desde P(s) hasta Y(s): Gper(s)
#    = Gptfm(s) / [1 + Gc(s)*Gptfm(s)]
# Observa que, con la librería control, la manera más directa de obtener
# esa misma forma es:
Gper_tf = ctl.feedback(Gptfm, Gc)       # Gptfm / [1 + Gptfm*Gc]
Gper_ss = ctl.ss(Gper_tf)              # Convertimos a EE para usarlo igual que Gref_ss

# Visualizar resultados
print("Gref(s) (espacio de estado) =", Gref_ss)
print("Gper(s) (espacio de estado) =", Gper_ss)


Gref(s) (espacio de estado) = <StateSpace>: sys[30]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']
States (2): ['x[0]', 'x[1]']

A = [[-1. -1.]
     [ 1. -2.]]

B = [[1.]
     [0.]]

C = [[0. 1.]]

D = [[0.]]

Gper(s) (espacio de estado) = <StateSpace>: sys[31]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']
States (2): ['x[0]', 'x[1]']

A = [[-3. -3.]
     [ 1.  0.]]

B = [[1.]
     [0.]]

C = [[1. 1.]]

D = [[0.]]



## Respuesta a preguntas
## 1. Polinomio Característico

El **polinomio característico** se obtiene al aplicar la ecuación determinantal:

\[
det(sI - A) = 0
\]

donde:

- **A** es la matriz de estado del sistema.
- **s** es la variable compleja.
- **I** es la matriz identidad.

Este polinomio resume las propiedades dinámicas del sistema, ya que sus raíces determinan el comportamiento en términos de estabilidad y respuesta transitoria.

## 2. Valores Propios

Los **valores propios** (o autovalores) son las soluciones de la ecuación:

\[
det(sI - A) = 0
\]

Estos valores indican:

- **Estabilidad:** El sistema es estable si todos los valores propios tienen partes reales negativas.
- **Respuesta Transitoria:** La magnitud y ubicación en el plano complejo determinan la rapidez y oscilaciones de la respuesta del sistema.

## 3. Tiempo de Estabilización y Sobre Nivel Porcentual

**Tiempo de Estabilización:** Es el tiempo que tarda la respuesta del sistema en alcanzar y permanecer dentro de una banda de tolerancia (por ejemplo, ±2% o ±5%) alrededor del valor final tras una entrada escalón. Un tiempo menor indica que el sistema alcanza rápidamente su estado estacionario.

**Sobre Nivel Porcentual (Percent Overshoot):** Es la medida del exceso máximo de la respuesta sobre el valor final deseado, expresado como porcentaje. Un sobre nivel alto indica oscilaciones significativas antes de estabilizarse, lo cual puede ser indeseable para la precisión y seguridad del sistema.


### 1. `tic()` / `toc()`
- **Función:** `tic()` inicia un cronómetro y `toc()` mide el tiempo transcurrido.
- **Uso:** Se usan juntos para medir el tiempo de ejecución de un bloque de código.

---

### 2. `time()`
- **Función:** Retorna el tiempo actual en segundos desde la época (1970).
- **Uso:** Útil para medir tiempos o registrar marcas temporales.

---

### 3. `tf(num, den)`
- **Función:** Crea una **función de transferencia** a partir del numerador y denominador.
- **Uso:** Representa sistemas lineales en el dominio de Laplace.

---

### 4. `ss(A, B, C, D)`
- **Función:** Crea una representación en **espacio de estados** del sistema.
- **Uso:** Para trabajar con sistemas en forma matricial (estado).

---

### 5. `zpk(z, p, k)`
- **Función:** Crea una función de transferencia a partir de **ceros, polos y ganancia**.
- **Uso:** Alternativa a `tf()` que usa la forma factorizada.

---

### 6. `tf2ss(system)`
- **Función:** Convierte una función de transferencia a espacio de estados.
- **Uso:** Facilita análisis y control en representación matricial.

---

### 7. `tf2zp(system)`
- **Función:** Convierte una función de transferencia a ceros, polos y ganancia.
- **Uso:** Permite ver estructura factorizada del sistema.

---

### 8. `ss2zp(system)`
- **Función:** Convierte un sistema en espacio de estados a ceros, polos y ganancia.
- **Uso:** Útil para análisis en forma factorizada.

---

### 9. `ss2tf(system)`
- **Función:** Convierte de espacio de estados a función de transferencia.
- **Uso:** Permite obtener la expresión simbólica del sistema.

---

### 10. `zp2ss(z, p, k)`
- **Función:** Convierte de ceros, polos y ganancia a espacio de estados.
- **Uso:** Para trabajar con el sistema en forma matricial.

---

### 11. `zp2tf(z, p, k)`
- **Función:** Convierte ceros, polos y ganancia a función de transferencia.
- **Uso:** Genera la forma racional (num/den) desde la factorizada.

---

### 12. `roots(p)`
- **Función:** Devuelve las **raíces de un polinomio** dado por sus coeficientes.
- **Uso:** Para hallar ceros o polos a partir de expresiones algebraicas.

---

### 13. `pole(system)`
- **Función:** Retorna los **polos** del sistema.
- **Uso:** Clave para determinar estabilidad y dinámica del sistema.

---

### 14. `zero(system)`
- **Función:** Retorna los **ceros** del sistema.
- **Uso:** Ayuda a analizar el comportamiento en frecuencia y la respuesta del sistema.

---

### 15. `poly(list_of_roots)`
- **Función:** Convierte una lista de raíces (ceros o polos) en un **polinomio**.
- **Uso:** Útil para construir funciones de transferencia a partir de ceros o polos conocidos.

---

### 16. `eig(A)`
- **Función:** Calcula los **valores propios** (eigenvalores) de una matriz.
- **Uso:** Se usa en representación en espacio de estados para estudiar la estabilidad del sistema.

---

### 17. `pzmap(system)`
- **Función:** Muestra un gráfico con los **polos y ceros** del sistema en el plano complejo.
- **Uso:** Útil para visualizar estabilidad y comportamiento dinámico.

---

### 18. `dcgain(system)`
- **Función:** Calcula la **ganancia en estado estacionario** (ganancia DC) del sistema.
- **Uso:** Muy útil en análisis de respuesta a entrada escalón.

---

### 19. `minreal(system)`
- **Función:** Simplifica el sistema eliminando **polos y ceros cancelables**.
- **Uso:** Reduce complejidad sin cambiar el comportamiento del sistema.

---

### 20. `subplot(nrows, ncols, index)`
- **Función:** Divide los plots en una sola ventana
- **Uso:** Divide la ventana de gráficos en subgráficos.

---

### 21. `feedback(sys1, sys2=1)`
- **Función:** Crea la función de transferencia en **lazo cerrado** a partir de `sys1` y `sys2`.
- **Uso:** Para modelar sistemas con realimentación (por defecto negativa).

---

### 22. `tfdata(system)`
- **Función:** Devuelve los **coeficientes** del numerador y denominador de una función de transferencia.
- **Uso:** Permite acceder y modificar directamente las expresiones algebraicas del sistema.
